<a href="https://colab.research.google.com/github/1ForrestWargo1/GDSC_Chess_Project/blob/main/wokring_yolo7_pose_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
p_map = {'nose':0,'left_foot': 16, "right_foot": 15,'left_eye':2,"right_shoulder":5,'left_shoulder':6,'left_hip':12,'left_hand':9,'left_ear':4}

In [ ]:
! git clone https://github.com/WongKinYiu/yolov7.git


Cloning into 'yolov7'...
remote: Enumerating objects: 998, done.
remote: Total 998 (delta 0), reused 0 (delta 0), pack-reused 998
Receiving objects: 100% (998/998), 69.77 MiB | 27.49 MiB/s, done.
Resolving deltas: 100% (465/465), done.


In [ ]:
%cd yolov7
!ls

/content/yolov7
cfg	detect.py  hubconf.py  models	  requirements.txt  tools	  utils
data	export.py  inference   paper	  scripts	    train_aux.py
deploy	figure	   LICENSE.md  README.md  test.py	    train.py


In [ ]:
! curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt -o yolov7-w6-pose.pt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  153M  100  153M    0     0  21.1M      0  0:00:07  0:00:07 --:--:-- 29.4M


In [ ]:
import torch
from torchvision import transforms

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_model():
    model = torch.load('yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

In [ ]:
def run_inference(image):
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (567, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 567, 960])
    if torch.cuda.is_available():
      image = image.half().to(device)
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 567, 960])
    with torch.no_grad():
      output, _ = model(image)
    return output, image


In [ ]:
def pose_estimation_video(filename,save_file = None):
    global standing_angle, standing_mean, standing_std
    set_standing_angle = len(standing_angle) == 0
    if set_standing_angle:
      print("SETTING STANDING ANGLE")
    else:
      print("CALCULATING CURRENT ANGLE")


    cap = cv2.VideoCapture(filename)
    # VideoWriter for saving the video
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    if not save_file:
      save_file =  'test_output.mp4'
    out = cv2.VideoWriter(save_file, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))
    count =0
    print('starting')
    while cap.isOpened():
        
        (ret, frame) = cap.read()
        count +=1
        #print(count)
        if ret == True:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            output, frame = run_inference(frame)
  

            frame = draw_keypoints(output, frame*0,set_standing_angle)
            frame = cv2.resize(frame, (int(cap.get(3)), int(cap.get(4))))
            out.write(frame)
            #cv2.imshow('Pose estimation', frame)
        else:
            break

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    if set_standing_angle:
      standing_angle = np.array(standing_angle)
      standing_mean = np.mean(standing_angle,axis = 0)
      standing_std = np.std(standing_angle,axis = 0)


    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
import numpy as np
def calc_shoulder_to_foot_dist(points):
  foot_points = np.array([points[p_map["left_foot"]*3:p_map["left_foot"]*3+3],points[p_map["right_foot"]*3:p_map["right_foot"]*3+3]])
  shoulder_points = np.array([points[p_map["left_shoulder"]*3:p_map["left_shoulder"]*3+3],points[p_map["right_shoulder"]*3:p_map["right_shoulder"]*3+3]])
  if sum(foot_points[:,2]) < 0.1 or sum(shoulder_points[:,2]) < 0.1:
    return [-1,-1,-1],[-1,-1],[-1,-1]

  foot_conf = foot_points[:,2]/sum(foot_points[:,2])
  shoulder_conf = shoulder_points[:,2]/sum(shoulder_points[:,2])

  avg_foot = foot_points[0,:2]*foot_conf[0] + foot_points[1,:2]*foot_conf[1]
  avg_shoulder = shoulder_points[0,:2]*shoulder_conf[0] + shoulder_points[1,:2]*shoulder_conf[1]
  return [sum(abs(avg_foot-avg_shoulder)),abs(avg_foot[0]-avg_shoulder[0]),abs(avg_foot[1]-avg_shoulder[1])],avg_foot,avg_shoulder

  



In [ ]:
def add_distance_chart(nimg,shoulder_to_foot_dist,idx):
      thickness = 5
      gap = thickness*2

      cv2.line(nimg, [0,0+3*idx*gap], [int(shoulder_to_foot_dist[0]),3*idx*gap], (255,0,255), thickness) 
      cv2.line(nimg, [0,gap+3*idx*gap], [int(shoulder_to_foot_dist[1]),gap+3*idx*gap], (0,0,255), thickness) 
      cv2.line(nimg, [0,2*gap+3*idx*gap], [int(shoulder_to_foot_dist[2]),2*gap+3*idx*gap], (255,0,0), thickness) 
      return nimg
def draw_posture_lines(nimg,standing_line,current_line):
  thickness = 5
  cv2.line(nimg, (100,100), np.array((100,100)+standing_line*[100,100]).astype(int),(0,0,0), thickness)
  cv2.line(nimg, (200,100), np.array((200,100)+current_line*[100,100]).astype(int),(0,0,0), thickness)





In [ ]:
standing_angle = []
standing_std = None
standing_mean = None

In [ ]:
standing_mean = np.mean(standing_angle,axis = 0)
standing_std = np.std(standing_angle,axis = 0)
print(standing_mean,standing_std)

nan nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
def draw_keypoints(output, image,set_standing_angle):
  global standing_angle
  output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
  
  with torch.no_grad():
        output = output_to_keypoint(output)
  #print("output after",output.shape,output)
  nimg = image[0].permute(1, 2, 0) * 255
  nimg = nimg.cpu().numpy().astype(np.uint8)
  nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
  #print('nimg',nimg.shape)


  for idx in range(output.shape[0]):
      plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
      shoulder_to_foot_dist, foot_point, shoulder_point = calc_shoulder_to_foot_dist(output[idx, 7:].T)
      if foot_point[0] == -1:
        continue
      #nimg = add_distance_chart(nimg,shoulder_to_foot_dist,idx)
      current_angle = calc_current_angle(foot_point,shoulder_point)
      if set_standing_angle:
        standing_angle.append(current_angle)
      else:
        angle_diff = (current_angle - standing_mean)**2
        print("angle diff",angle_diff)
        draw_posture_lines(nimg,standing_mean,current_angle)
        if angle_diff[0] > standing_std[0] or angle_diff[1] > standing_std[1]:
          cv2.line(nimg, [300,30], [300,60], (0,255,0), 10) 
        else:
          cv2.line(nimg, [300,30], [300,60], (255,255,255), 10) 


    
  
  
    
  





  #calc standing from example video
  #compare to laying down cangle
  #comapre to sitting angle adding waist -- these weights could be learned (?)


      #print("test",shoulder_to_foot_dist)
      #print("skeleton point", idx,output[idx, 7:].T, len(output[idx, 7:]) )

  return nimg

In [ ]:
def calc_current_angle(foot_point,shoulder_point):
  #print(foot_point,shoulder_point)
  vec = abs(foot_point-shoulder_point)/sum(abs(foot_point-shoulder_point))
  #print(vec)
  return vec



In [ ]:
standing_angle = []
standing_std = None
standing_mean = None
pose_estimation_video('/content/drive/MyDrive/Alarm/innovation week/walking.mov','/content/drive/MyDrive/Alarm/innovation week/save/standing_output.mp4')


SETTING STANDING ANGLE


/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
pose_estimation_video('/content/drive/MyDrive/Alarm/innovation week/up_down.mp4','/content/drive/MyDrive/Alarm/innovation week/save/up_down_black_no_dist.mp4')


CALCULATING CURRENT ANGLE
starting
angle diff [ 0.00010379  0.00010379]
angle diff [ 5.3357e-06  5.3357e-06]
angle diff [ 9.5411e-07  9.5411e-07]
angle diff [ 3.1444e-05  3.1444e-05]
angle diff [ 0.00016688  0.00016688]
angle diff [ 5.8344e-05  5.8344e-05]
angle diff [ 6.0969e-05  6.0969e-05]
angle diff [ 0.00017634  0.00017634]
angle diff [ 3.4659e-05  3.4659e-05]
angle diff [ 1.0227e-05  1.0227e-05]
angle diff [ 0.00013907  0.00013907]
angle diff [ 0.00083404  0.00083404]
angle diff [  0.0016234   0.0016234]
angle diff [  0.0019617   0.0019617]
angle diff [  0.0017826   0.0017826]
angle diff [  0.0012314   0.0012314]
angle diff [  0.0010834   0.0010834]
angle diff [  0.0012151   0.0012151]
angle diff [  0.0015227   0.0015227]
angle diff [  0.0018103   0.0018103]
angle diff [ 0.00070284  0.00070284]
angle diff [ 1.3402e-05  1.3402e-05]
angle diff [ 0.00014795  0.00014795]
angle diff [   0.001187    0.001187]
angle diff [  0.0031541   0.0031541]
angle diff [  0.0051821   0.0051821]
ang

In [ ]:
print(standing_mean)

None


In [ ]:
test = torch.load("/content/drive/MyDrive/Alarm/save/1.pt")


In [ ]:
test.shape

Take two

In [ ]:
def pose_video(frame):
    mapped_img = frame.copy()
    # Letterbox resizing.
    img = letterbox(frame, input_size, stride=64, auto=True)[0]
    print(img.shape)
    img_ = img.copy()
    # Convert the array to 4D.
    img = transforms.ToTensor()(img)
    # Convert the array to Tensor.
    img = torch.tensor(np.array([img.numpy()]))
    # Load the image into the computation device.
    img = img.to(device)
     
    # Gradients are stored during training, not required while inference.
    with torch.no_grad():
        t1 = time.time()
        output, _ = model(img)
        t2 = time.time()
        fps = 1/(t2 - t1)
        output = non_max_suppression_kpt(output, 
                                         0.25,    # Conf. Threshold.
                                         0.65,    # IoU Threshold.
                                         nc=1,   # Number of classes.
                                         nkpt=17, # Number of keypoints.
                                         kpt_label=True)
         
        output = output_to_keypoint(output)
 
    # Change format [b, c, h, w] to [h, w, c] for displaying the image.
    nimg = img[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
 
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
         
    return nimg, fps

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt


In [ ]:
import matplotlib.pyplot as plt
import torch
import cv2
import numpy as np
import time
 
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weigths = torch.load('yolov7-w6-pose.pt')
model = weigths['model']
model = model.half().to(device)
_ = model.eval()
 
video_path = '/content/drive/MyDrive/Alarm/walking.mp4'

In [ ]:
cap = cv2.VideoCapture(video_path)
if (cap.isOpened() == False):
  print('Error while trying to read video. Please check path again')
 
# Get the frame width and height.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
 
# Pass the first frame through `letterbox` function to get the resized image,
# to be used for `VideoWriter` dimensions. Resize by larger side.
vid_write_image = letterbox(cap.read()[1], (frame_width), stride=64, auto=True)[0]
resize_height, resize_width = vid_write_image.shape[:2]
 
save_name = f"{video_path.split('/')[-1].split('.')[0]}"
# Define codec and create VideoWriter object .
out = cv2.VideoWriter(f"{save_name}_keypoint.mp4",
                    cv2.VideoWriter_fourcc(*'mp4v'), 30,
                    (resize_width, resize_height))
 
 
frame_count = 0 # To count total frames.
total_fps = 0 # To get the final frames per second.

In [ ]:
while(cap.isOpened):
  # Capture each frame of the video.
  ret, frame = cap.read()
  if ret:
      orig_image = frame
      image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
      image = letterbox(image, (int(cap.get(3))), stride=64, auto=True)[0]
      image_ = image.copy()
      image = transforms.ToTensor()(image)
      image = torch.tensor(np.array([image.numpy()]))
      image = image.to(device)
      image = image.half()
 
      # Get the start time.
      start_time = time.time()
      with torch.no_grad():
          output, _ = model(image)
        # Get the end time.
      end_time = time.time()
      # Get the fps.
      fps = 1 / (end_time - start_time)
      # Add fps to total fps.
      total_fps += fps
      # Increment frame count.
      frame_count += 1
 
      output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
      output = output_to_keypoint(output)
      nimg = image[0].permute(1, 2, 0) * 255
      nimg = nimg.cpu().numpy().astype(np.uint8)
      nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
      for idx in range(output.shape[0]):
          plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
 
          # Comment/Uncomment the following lines to show bounding boxes around persons.
          xmin, ymin = (output[idx, 2]-output[idx, 4]/2), (output[idx, 3]-output[idx, 5]/2)
          xmax, ymax = (output[idx, 2]+output[idx, 4]/2), (output[idx, 3]+output[idx, 5]/2)
          cv2.rectangle(
              nimg,
              (int(xmin), int(ymin)),
              (int(xmax), int(ymax)),
              color=(255, 0, 0),
              thickness=1,
              lineType=cv2.LINE_AA
          )
 
      # Write the FPS on the current frame.
      cv2.putText(nimg, f"{fps:.3f} FPS", (15, 30), cv2.FONT_HERSHEY_SIMPLEX,
                  1, (0, 255, 0), 2)
      # Convert from BGR to RGB color format.
      cv2.imshow('image', nimg)
      out.write(nimg)
      # Press `q` to exit.
      if cv2.waitKey(1) & 0xFF == ord('q'):
          break
  else:
      break
# Release VideoCapture().
cap.release()
# Close all frames and video windows.
cv2.destroyAllWindows()
# Calculate and print the average FPS.
avg_fps = total_fps / frame_count
print(f"Average FPS: {avg_fps:.3f}")

Take 3

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!git clone https://github.com/RizwanMunawar/yolov7-pose-estimation.git
!cd yolov7-pose-estimation
!pip install --upgrade pip
!pip install -r requirements.txt


In [ ]:
!cd yolov7-pose-estimation; python pose-estimate.py --source "/content/drive/MyDrive/Alarm/IMG_7970.MOV" --device cpu

In [ ]:
#!cd yolov7-pose-estimation;curl -L https://github.com/RizwanMunawar/yolov7-pose-estimation -o yolov7-w6-pose.pt
!cd yolov7-pose-estimation; wget https://github.com/RizwanMunawar/yolov7-pose-estimation


In [ ]:
!cp /content/drive/MyDrive/Alarm/yolov7-w6-pose.pt /content/yolov7/yolov7-pose-estimation/